<a href="https://colab.research.google.com/github/yn2a75/benkyokai/blob/main/pdf2text_ExtractCompanyName_RenameFle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#必要なライブラリをインストール
! pip install pdfminer.six
! pip install pandas
! pip install openpyxl
! pip install xlwt
! pip install xlsxwriter

     |████████████████████████████████| 5.6 MB 26.4 MB/s 
     |████████████████████████████████| 3.6 MB 53.5 MB/s 
     |████████████████████████████████| 149 kB 21.0 MB/s 


### PDFファイルからテキストを出力する関数を作成

In [49]:
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
import os
from io import StringIO

#PDFからテキストを出力する関数を定義
def convert_pdf2txt(filename):
  pdf_file = os.path.join(pdffiles_folder, filename)
  #バイナリファイルを読み込むためにファイルを開く('rb'を指定)
  fp = open(pdf_file, 'rb')
    
  outfp = StringIO()
        
  rmgr = PDFResourceManager()
  lprms = LAParams()
  device = TextConverter(rmgr, outfp, laparams=lprms)
  iprtr = PDFPageInterpreter(rmgr, device)
    
  for page in PDFPage.get_pages(fp):
    iprtr.process_page(page)

#テキスト内容を取得し、改行とスペースを削除
  text = outfp.getvalue()
  text=text.replace("\n", "")
  text=text.replace("   ", "\n　")
  #text=text.replace(" ", "")

#テキスト内容を返す
  return text

In [3]:
#GoogleDriveをマウント
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [50]:
#PDFを保管しているフォルダのpathを指定

pdffiles_folder=r"/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/PDFfiles"

#テキストを格納する空のリストを作成
text_list=[]


filelist = os.listdir(pdffiles_folder)
filelist = [f for f in filelist if '.pdf' in f]
pdfpaths = [os.path.join(pdffiles_folder,f) for f in filelist if '.pdf' in f]

#リストにPDFのテキストを追加
for filename in filelist:
  text_list.append(convert_pdf2txt(filename))

In [51]:
len(text_list)
len(filelist)
len(pdfpaths)

5

### テキストから会社名を抜き出す
ginzaとspacyを使う

In [52]:
#ライブラリをインストール
!pip install -U ginza
!pip install -U ginza ja-ginza

In [30]:
#spacyをインポートしてインスタンス化
import spacy
from spacy import displacy
nlp = spacy.load('ja_ginza')

OrderDictを使ってリストから、順序を維持したうえで重複を削除したリストを作成する
https://www.delftstack.com/ja/howto/python/remove-duplicates-from-list-python/

In [31]:
#OrderDictをインポート
from collections import OrderedDict

### テキストから会社名を返す関数を作成

In [53]:
#テキストから会社名を返す関数をつくる

def Text2CompanyName(text):
  doc=nlp(text)
  #空のリストを準備
  company_names_=[]
#固有表現ラベルがCompanyをリストに追加
  for ent in doc.ents: 
    if ent.label_ =='Company':
      company_names_.append(ent.text) 
      company_names=list(OrderedDict.fromkeys(company_names_))
  if len(company_names_)==0:
    return ["なし"]
  else:
    return company_names

### 参考
listが空かどうか判定する方法
https://qiita.com/yonedaco/items/d0f65ca3dad2e085a51d

### 用意した関数を使い、テキストから会社名を抽出して新たなリストを作成する

In [54]:
#companyの名前を入れる空のリストを準備
companyNames1=[]
for company in text_list:
    companyNames1.append(Text2CompanyName(company))

### 2次元リストの各行の0番目値を取得して新たにリストを作る  
https://ja.stackoverflow.com/questions/47331/python%E3%81%A7%E4%BA%8C%E6%AC%A1%E5%85%83%E9%85%8D%E5%88%97%E3%81%AE%E4%B8%AD%E3%81%AE%E5%90%84%E8%A6%81%E7%B4%A0%E3%81%AEn%E7%95%AA%E7%9B%AE%E3%81%A0%E3%81%91%E3%82%92%E5%8F%96%E3%82%8A%E5%87%BA%E3%81%97%E3%81%A6-%E8%A6%81%E7%B4%A0%E3%81%A8%E3%81%97%E3%81%A6%E4%B8%A6%E3%81%B9%E3%81%9F%E3%81%84

In [55]:
#CompanyName1の各行の0番目の値をCompanyNames2に格納
companyNames2=list(map(lambda companyName1:companyName1[0],companyNames1))

### pandasでエクセル表のもととなる一覧表を作成
https://www.delftstack.com/ja/howto/python-pandas/pandas-create-dataframe-from-list/

In [56]:
#pandasをインポート
import pandas as pd

In [57]:
#データフレームを作成
df=pd.DataFrame(list(zip(pdfpaths,filelist,text_list,companyNames1,companyNames2)),columns=["path","filename","text","companynames","companyname"])

In [58]:
df

,path,filename,text,companynames,companyname
0,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220311504284.pdf,各 位2022年3月11日会社名\n 住友商事株式会社代表者名 代表取締役 社長執行役員...,"[住友商事株式会社, (株)現分掌, (株)執行役員リース, (株) ２．, ＳＣＳＫ, (...",住友商事株式会社
1,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220301598700.pdf,各 位 2022年３月１日 会 社 名 テ 代表者名 代表取締役社長 ラ 株 式 会 ...,"[定時株主総会, アジアンブロードキャピタル・インベストメント, 株式会社マリオット]",定時株主総会
2,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220301598613.pdf,各 位\n 2022 年３月１日\n 会社名 明治機械株式会社 代表者名 取締役社...,[なし],なし
3,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220301598563.pdf,2022 年 3 月 1 日 会 社 名 代表者名 株式会社網屋 代表取締役会長 伊藤 ...,[代表者名 株式会社],代表者名 株式会社
4,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220301597890.pdf,2022 年 3 月 1 日 東 京 都 千 代 田 区 麹 町 一 丁 目 4 番 地 松...,[松井証券],松井証券


In [63]:
#データフレームをエクセル出力
output_excel_path=pdffiles_folder + "list.xlsx"
df.to_excel(output_excel_path , engine='xlsxwriter')

### Pandasでエクセルファイルに書き出そうとすると表示される「IllegalCharacterError」を回避
pandasの書き出しエンジンをxlsxwriterに変更する  
df.to_excel("test.xlsx", engine='xlsxwriter')  
https://karupoimou.hatenablog.com/entry/2019/09/29/235528

### PDFのファイル名を変更し、別フォルダに移動して保存する。
https://note.nkmk.me/python-shutil-move/

In [64]:
#会社名を確認したエクセルをデータフレームに読み込む
df2 = pd.read_excel(output_excel_path)

In [65]:
#新しいファイル名を含めた移動先のパスを指定
df2["topath"]=pdffiles_folder + "/" + "processed" + "/" +df2["companyname"] + df2["filename"] 

In [66]:
#ファイルを移動するためのモジュールをインポート 
import shutil
import os

In [67]:
#if文でファイルが存在するかを確認した上で、ファイルを移動する
for path, topath in zip(df2["path"],df2["topath"]):
    if os.path.exists(path):
        new_path=shutil.move(path,topath)
    else:
        print("file in not exist")


file in not exist
file in not exist
file in not exist
